In [ ]:
'''
This notebook is an attempt at reproducing the CCZ gate of Figure 4. in Reference [6]
'''

In [1]:
import math

import perceval as pcvl
import perceval.components.unitary_components as comp

import qiskit
from perceval.converters import QiskitConverter
from perceval.components import catalog

In [2]:
# Gates
CNOT1 = pcvl.catalog['postprocessed cnot'].build_processor()

T = pcvl.PS(math.pi/4)
T_DAG = pcvl.PS(-math.pi/4) 

# Simulation config
logical_qubits = 3
source = pcvl.Source(emission_probability=0.40, multiphoton_component=0.01)
QPU = pcvl.Processor("SLOS", logical_qubits*2, source)

# Qiskit config
qc = qiskit.QuantumCircuit(logical_qubits)

![ccz](img/ccz.png)

In [3]:
qc.cx(1, 2)
qc.tdg(2)
qc.cx(0, 2)
qc.t(2)
qc.cx(1, 2)
qc.tdg(2)
qc.cx(0, 2)
qc.t([1, 2])
qc.cx(0, 1)
qc.t(0)
qc.tdg(1)
qc.cx(0, 1)

In [4]:
qiskit_convertor = QiskitConverter(catalog)
QPU = qiskit_convertor.convert(qc)

In [5]:
from itertools import product

# Generar todas las permutaciones posibles de 0 y 1 en una cadena de 3 dígitos
all_permutations = [''.join(map(str, perm)) for perm in product([0, 1], repeat=3)]

# Crear un diccionario donde la clave y el valor son la misma cadena
permutations_dict = {perm: perm for perm in all_permutations}

print(permutations_dict)

{'000': '000', '001': '001', '010': '010', '011': '011', '100': '100', '101': '101', '110': '110', '111': '111'}


In [25]:
states = {
     pcvl.BasicState([1, 0, 1, 0, 1, 0]): "000",
     pcvl.BasicState([1, 0, 1, 0, 0, 1]): "001",
     pcvl.BasicState([1, 0, 0, 1, 1, 0]): "010",
     pcvl.BasicState([0, 1, 1, 0, 1, 0]): "100",
     pcvl.BasicState([0, 1, 1, 0, 0, 1]): "101",
     pcvl.BasicState([1, 0, 0, 1, 0, 1]): "011",
     pcvl.BasicState([0, 1, 0, 1, 1, 0]): "110",
     pcvl.BasicState([0, 1, 0, 1, 0, 1]): "111",
}

ca = pcvl.algorithm.Analyzer(QPU, states)
ca.compute(expected={'000': '000', '001': '001', '010': '010', '011': '011', '100': '100', '101': '101', '110': '110', '111': '111'})

# Testing circuit
QPU.with_input(pcvl.BasicState([
     0, 1, # 1
     0, 1, # 1
     0, 1  # 1
]))

pcvl.pdisplay(ca)

print(dir(ca))
print(ca.distribution)

print(f"performance = {pcvl.simple_float(ca.performance)[1]}, fidelity = {ca.fidelity*100}%")

['_MAX_SHOTS_NAMED_PARAM', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_compatibility', '_distribution', '_mapping', '_max_shots', '_processor', '_sampler', 'col', 'compute', 'default_job_name', 'distribution', 'error_rate', 'fidelity', 'input_states_list', 'output_states_list', 'performance']
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.

In [7]:
output_distribution=QPU.probs()["results"]

pcvl.pdisplay(output_distribution, max_v=10)

# Psi =  1/sqrt(2) |00> + 1/sqrt(2) |00> - 1/sqrt(2) |00>
#|111> -> |111>
